We have two possible modelling questions here:
1. does $x$ come from a double hit? Here $x$ is two dimensional. We treat this as a logistic regression problem: $y = \sigma(f(s, t))$ with y being 1 if the datapoint is double-hit or 0 if single hit. We dont observe y though as of now, so this won't work. Maybe train from simulations and predict from real data? Is there a dataset for this?
2. is $x$ high or low fitness? classify datapoints based on $s$ only, 1D. Then, we can show data on 2D plot, but I guess we wont learn much from the 2D plot except that high $s$ values are rare for low $t_0$.

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path
import matplotlib as mpl
import pandas as pd
import socket
import numpy as np
import seaborn as sns
from scipy.stats import multivariate_normal, spearmanr
from matplotlib.patches import Ellipse
from matplotlib import font_manager
import matplotlib as mpl
from scipy.stats import chi2
from sklearn.mixture import BayesianGaussianMixture, GaussianMixture

if socket.gethostname() == "LAPTOP-CEKCHJ4C":
    font_dirs = [Path("~").expanduser() / ".local/share/fonts/otf/TexGyreHeros/"]
else:
    font_dirs = [Path("~").expanduser() / ".local/fonts/TeX-Gyre-Heros/"]
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)
for font_file in font_files:
    font_manager.fontManager.addfont(font_file)
mpl.rcParams["font.family"] = ["TeX Gyre Heros", "sans-serif"]

palette = "Dark2"
colors = mpl.colormaps[palette].colors

In [ ]:
data = pd.read_csv("variantTrajectoryEstimates.csv")  # , usecols=["sMax", "t0Max"])
data.plot(x="sMax", y="t0Max", ls="", marker=".")  # TODO: ask why s>1

In [ ]:
data

In [ ]:
def plot_gaussian(ax, mu, cov, c):
    # need this to rotate the Gaussians
    # eigen-decomposition of covariance
    vals, vecs = np.linalg.eigh(cov)
    order = vals.argsort()[::-1]
    vals, vecs = vals[order], vecs[:, order]

    for p in (0.5, 0.9, 0.99):  # probability mass
        s = chi2.ppf(p, df=2)  # chi-square quantile
        width, height = 2 * np.sqrt(vals * s)  # radii along eigenvectors
        angle = np.degrees(np.arctan2(vecs[1, 0], vecs[0, 0]))
        e = Ellipse(
            xy=mu,
            width=width,
            height=height,
            angle=angle,
            fill=False,
            linewidth=1.5,
            color=c,
        )
        ax.add_patch(e)
        # ax.annotate(f"{int(p*100)}%", (mu[0]+width/2, mu[1]))

    # ax.set_aspect("equal")
    # ax.set_title("2D Gaussian confidence ellipses")
    return ax


def plot_nice(ax, means, covs, color, labels=None):
    for m, sigma, c in zip(means, covs, color):
        ax = plot_gaussian(ax, m, sigma, c=c)
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 70])

In [ ]:
k_components = 3

fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(6, 3), layout="tight")
sns.scatterplot(data=data, x="sMax", y="t0Max", ax=ax[0], color="#7570b3")

gmm = BayesianGaussianMixture(
    n_components=k_components,
    covariance_type="full",
    tol=0.001,
    reg_covar=1e-06,
    max_iter=1000,
    n_init=1,
    init_params="random_from_data",
    weight_concentration_prior_type="dirichlet_process",
    # weight_concentration_prior=0.01,
    mean_precision_prior=None,
    mean_prior=None,
    degrees_of_freedom_prior=5,
    covariance_prior=None,
    random_state=None,
    warm_start=False,
    verbose=1,
    verbose_interval=10,
)

labels = gmm.fit_predict(data[["sMax", "t0Max"]].to_numpy())
print(gmm.weights_)
sns.scatterplot(data=data, x="sMax", y="t0Max", hue=labels, ax=ax[1], palette=palette)
plot_nice(ax[1], gmm.means_, gmm.covariances_, colors, labels)
ax[1].legend().set_visible(False)
plt.savefig("./dirichlet_3_components.png", dpi=700)
plt.show()

In [ ]:
# PRIORS
n_features = 2  # 2D problem
k_components = 2

# means
# 1st component of the mixture
mean_s_1, mean_t_1 = 0.2, 40
mean_prior_1 = np.array([mean_s_1, mean_t_1], dtype=float)
# 2nd component of the mixture
mean_s_2, mean_t_2 = 0.4, 60
mean_prior_2 = np.array([mean_s_2, mean_t_2], dtype=float)
# combine them
mean_prior = np.array([mean_prior_1, mean_prior_2], dtype=float)
assert mean_prior.shape == (k_components, n_features), mean_prior.shape

# covariances
# 1st component of the mixture
var_s_1, var_t_1, cov_s_t_1 = 0.01, 100, 0.2  # TODO
cov_prior_1 = np.array([[var_s_1, cov_s_t_1], [cov_s_t_1, var_t_1]], dtype=float)
assert cov_prior_1.shape == (2, 2), cov_prior_1.shape
# 2nd component of the mixture
var_s_2, var_t_2, cov_s_t_2 = 0.01, 30, 0.2  # TODO
cov_prior_2 = np.array([[var_s_2, cov_s_t_2], [cov_s_t_2, var_t_2]], dtype=float)
assert cov_prior_2.shape == (2, 2), cov_prior_2.shape
# combine
cov_prior = np.array([cov_prior_1, cov_prior_2])
assert cov_prior.shape == (k_components, n_features, n_features), cov_prior.shape

fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(6, 3), layout="tight")
sns.scatterplot(data=data, x="sMax", y="t0Max", ax=ax[0], color="#7570b3")
plot_nice(ax[0], [mean_prior_1, mean_prior_2], [cov_prior_1, cov_prior_2], colors)
ax[0].set_title("Initialisation")
gmm = GaussianMixture(
    n_components=k_components,
    covariance_type="full",
    reg_covar=1e-09,
    tol=1e-6,
    max_iter=1000,
    weights_init=[0.9, 0.1],
    means_init=mean_prior,
    precisions_init=np.linalg.inv(cov_prior),  # not the best
    random_state=None,
    warm_start=False,
    verbose=2,
    verbose_interval=10,
)

labels = gmm.fit_predict(data[["sMax", "t0Max"]].to_numpy())
print(gmm.weights_init)
print(gmm.weights_)
sns.scatterplot(data=data, x="sMax", y="t0Max", hue=labels, ax=ax[1], palette=palette)
plot_nice(ax[1], gmm.means_, gmm.covariances_, colors, labels)
ax[1].legend().set_visible(False)
ax[1].set_title("Inference result")
plt.savefig("./gmm_2_components.png", dpi=700)
plt.show()

In [ ]:
gmm = BayesianGaussianMixture(
    n_components=100,
    covariance_type="full",
    tol=0.001,
    reg_covar=1e-06,
    max_iter=1000,
    n_init=1,
    init_params="random_from_data",
    weight_concentration_prior_type="dirichlet_process",
    # weight_concentration_prior=0.01,
    mean_precision_prior=None,
    mean_prior=None,
    degrees_of_freedom_prior=5,
    covariance_prior=None,
    random_state=None,
    warm_start=False,
    verbose=1,
    verbose_interval=10,
)

labels = gmm.fit_predict(data[["sMax", "t0Max"]].to_numpy())
print(gmm.weights_)
fig, ax = plt.subplots(figsize=(5, 4), layout="tight")
plot_nice(ax, gmm.means_, gmm.covariances_, colors, labels)
ax = sns.scatterplot(data=data, x="sMax", y="t0Max", hue=labels, ax=ax, palette=palette)
plt.savefig("./dirichlet_100_components.png", dpi=700)
plt.show()

In [ ]:
gmm = GaussianMixture(
    n_components=2,
    covariance_type="full",
    reg_covar=1e-09,
    tol=1e-6,
    max_iter=1000,
    # weights_init=[0.9, 0.1],
    # means_init=mean_prior,
    # precisions_init=np.linalg.inv(cov_prior),  # not the best
    random_state=None,
    warm_start=False,
    verbose=2,
    verbose_interval=10,
)

labels = gmm.fit_predict(data["sMax"].to_numpy().reshape(-1, 1))
data["ClusterHigh"] = labels
data["ClusterHigh"] = data["ClusterHigh"].astype(bool)
decision_boundary = data.loc[data.ClusterHigh, "sMax"].min()

fig, ax = plt.subplots(
    1, 2, sharex=True, sharey=False, figsize=(7.5, 3), layout="tight"
)
ax[0].axvline(x=decision_boundary, color="black", linewidth=1.5, ls="--")
sns.scatterplot(
    data=data, x="sMax", y="t0Max", hue="ClusterHigh", ax=ax[1], palette=palette
)
# ax[1].legend(title=f"ClusterHigh\n  s>{decision_boundary:.4f}")
sns.histplot(
    data=data["sMax"],
    stat="density",
    ax=ax[0],
    color="#7570b3",
    binwidth=0.025,
    label="Data distribution",
)
ax[1].set_xlim([0, 1])
ax[1].axvline(x=decision_boundary, color="black", linewidth=1.5, ls="--")
ax[1].set_xlabel(r"Clone fitness $s$")
ax[1].set_ylabel(r"Age at clone arrival $t_0$")
ax[0].set_xlabel(r"Clone fitness $s$")
ax[1].legend(title=r"$s>s_T$")
for mu, var, c, l in zip(
    gmm.means_,
    gmm.covariances_,
    mpl.colormaps[palette].colors,
    ["Low fitness distribution\n", "High fitness distribution\n"],
):
    s = np.random.normal(mu, np.sqrt(var.ravel()), 10000)
    ax[0].hist(
        s,
        30,
        density=True,
        histtype="step",
        color=c,
        align="mid",
        label=l + r"$\mathcal{N}$" + rf"$(\mu={mu[0]:.2f}, \sigma^2={var[0][0]:.3f})$",
    )
ax[0].legend(fontsize="small")
ax[0].text(
    decision_boundary + decision_boundary * 0.1, 4, f"$s={decision_boundary:.4f}$"
)
ax[1].legend(
    fontsize="small",
    labels=["Low fitness clones\n$s<$0.2255", "High fitness clones\n$s\\geq$0.2255"],
)
ax[0].set_ylim([0, 15])
ax[0].set_yticks([0, 5, 10, 15])
plt.savefig("./SI GMM.jpg", dpi=700)
plt.show()

In [ ]:
sns.pairplot(
    data[["sMax", "t0Max"]],
)

In [ ]:
sns.pairplot(data, hue="ClusterHigh")

In [ ]:
data.loc[data.ClusterHigh, "pid"].value_counts().hist()

In [ ]:
data.loc[~data.ClusterHigh, "gene"].value_counts()

In [ ]:
data.loc[data.ClusterHigh, "gene"].value_counts()

In [ ]:
data.loc[data.ClusterHigh, "gene"].value_counts().hist()

In [ ]:
spearmanr(data[["sMax", "t0Max"]])  # reject H0 -> correlation, stat>0 so positive corr